In [161]:
import idx2numpy
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import random as rand
np.set_printoptions(suppress=True)

x_digit = idx2numpy.convert_from_file('data/train-images-idx3-ubyte')
y_digit = idx2numpy.convert_from_file('data/train-labels-idx1-ubyte')

print(x_digit.shape, y_digit.shape)

(60000, 28, 28) (60000,)


In [162]:
### Generate train and test data using mnist set with NUM_SIZE digits
NUM_SIZE = 6

image_dict = [[] for i in range(10)]
for digit,image in zip(y_digit, x_digit):
    image_dict[digit].append(image)


def gen_test_data(num_data):
    x_data, y_data = np.zeros((num_data, NUM_SIZE, 28, 28)), np.zeros((num_data, NUM_SIZE))
    for i in range(num_data):
        for k in range(NUM_SIZE):
            rand_digit = rand.randint(0,9)
            rand_img = rand.choice(image_dict[rand_digit])
            y_data[i,k] = rand_digit
            x_data[i,k] = rand_img
    return x_data.reshape(num_data, NUM_SIZE * 28, -1), y_data

x_train, y_train = gen_test_data(100000)
x_test, y_test = gen_test_data(10000)
x_test_check = x_test
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(100000, 168, 28) (100000, 6) (10000, 168, 28) (10000, 6)


In [163]:
y_test = tf.one_hot(y_test, 10).numpy()
y_train = tf.one_hot(y_train, 10).numpy()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
print(type(x_train), type(x_test), type(y_train), type(y_test))

(100000, 168, 28) (100000, 6, 10) (10000, 168, 28) (10000, 6, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [176]:
input_layer = tf.keras.Input(shape = (NUM_SIZE*28, 28))
_ = tf.keras.layers.Flatten()(input_layer) 
_ = tf.keras.layers.Dense(4096)(_)
_ = tf.keras.layers.Dropout(rate=0.1)(_)
common_layer = tf.keras.layers.Dense(1024)(_)

#One output layer per digit
output_layers = {}
output_loss_fns = {}
output_metrics = {}
y_trains = {}
y_tests = {}
for op_l in range(NUM_SIZE):
    layer_name = "output_digit_" + str(op_l)
    _ = tf.keras.layers.Dense(512)(common_layer)
    output_layers[op_l] = tf.keras.layers.Dense(10, activation = 'softmax', name=layer_name)(_)
    output_loss_fns[op_l] = tf.keras.losses.CategoricalCrossentropy()
    output_metrics[op_l] = 'accuracy'
    y_trains[op_l] = y_train[:,op_l,:]
    y_tests[op_l] = y_test[:,op_l,:]

In [177]:
model = tf.keras.Model(inputs=input_layer, outputs = output_layers, name="digit_recognizer")
model.compile(optimizer="Adam",
             loss = output_loss_fns,
             metrics = ['accuracy'] )
#model.summary()

In [ ]:
model.fit(x = x_train, y = y_trains, epochs=3, batch_size=500)

Epoch 1/3
200/200 [==============================] - 40s 186ms/step - loss: 10501.1774 - output_digit_0_loss: 1957.5518 - output_digit_1_loss: 1682.9938 - output_digit_2_loss: 1707.1506 - output_digit_3_loss: 1729.9518 - output_digit_4_loss: 1729.4393 - output_digit_5_loss: 1694.0899 - output_digit_0_accuracy: 0.5275 - output_digit_1_accuracy: 0.5232 - output_digit_2_accuracy: 0.5368 - output_digit_3_accuracy: 0.5450 - output_digit_4_accuracy: 0.5427 - output_digit_5_accuracy: 0.5494
Epoch 2/3
141/200 [====================>.........] - ETA: 10s - loss: 191.1585 - output_digit_0_loss: 32.2320 - output_digit_1_loss: 33.1343 - output_digit_2_loss: 32.3089 - output_digit_3_loss: 31.9948 - output_digit_4_loss: 32.4043 - output_digit_5_loss: 29.0842 - output_digit_0_accuracy: 0.8785 - output_digit_1_accuracy: 0.8786 - output_digit_2_accuracy: 0.8766 - output_digit_3_accuracy: 0.8798 - output_digit_4_accuracy: 0.8737 - output_digit_5_accuracy: 0.8835

In [172]:
model.evaluate(x=x_test, y=y_tests)

313/313 [==============================] - 36s 115ms/step - loss: 147.3889 - output_digit_0_loss: 10.6924 - output_digit_1_loss: 11.6446 - output_digit_2_loss: 11.0336 - output_digit_3_loss: 11.8417 - output_digit_4_loss: 9.9892 - output_digit_5_loss: 11.3214 - output_digit_0_accuracy: 0.8795 - output_digit_1_accuracy: 0.8797 - output_digit_2_accuracy: 0.8841 - output_digit_3_accuracy: 0.8769 - output_digit_4_accuracy: 0.8846 - output_digit_5_accuracy: 0.8817


[147.38893127441406,
 10.692400932312012,
 11.644603729248047,
 11.033641815185547,
 11.841721534729004,
 9.9891939163208,
 11.321382522583008,
 0.8794999718666077,
 0.8797000050544739,
 0.8841000199317932,
 0.8769000172615051,
 0.8845999836921692,
 0.8816999793052673]